In [1]:
from utils.common import download_multiple_images, iter_posts, load_data,save_data
from utils.extract_faces import FaceExtractor
import cv2
from utils.regexes import extract_gender_age, extract_rating
from config import SAVE_EVERY, DATA_PATH, IMG_PATH


Version 7.4.0 of praw is outdated. Version 7.5.0 was released Sunday November 14, 2021.


In [2]:
face_ratings = [
    "truerateme",
    "Rateme",
    "ratemyface",
]


In [3]:
subreddit = "Rateme"

In [4]:
image_path = IMG_PATH / subreddit
image_path.mkdir(exist_ok=True, parents=True)
data_path = DATA_PATH / f"{subreddit}.json"

face_extractor = FaceExtractor.default(additional_modules=["genderage"])
data = load_data(data_path)


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/idk/.insightface/models/buffalo_m/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/idk/.insightface/models/buffalo_m/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/idk/.insightface/models/buffalo_m/det_2.5g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/idk/.insightface/models/buffalo_m/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/idk/.insightface/models/buffalo_m/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [5]:
posts = iter_posts(subreddit, 5_000)


In [10]:
for j, post in enumerate(posts):
    if (
        post.id in data
        or post.removed_by_category
        or post.num_comments == 0
        or post.locked
    ):
        continue
    try:
        images = download_multiple_images(post)
    except Exception:
        continue

    if not images:
        continue

    gender, age = extract_gender_age(post.title)

    detections, faces, urls = face_extractor(
        images, additional_modules=(["genderage"] if not (gender and age) else [])
    )

    if not detections:
        continue

    ratings = []

    for comment in post.comments:
        rating = extract_rating(comment.body)
        ratings.append(
            dict(
                user=getattr(comment.author, "name", None),
                value=rating,
                body=comment.body,
            )
        )

    post_data = dict(
        gender=gender,
        age=age,
        urls=urls,
        detections=detections,
        title=post.title,
        score=post.score,
        author=getattr(post.author, "name", None),
        ratings=ratings,
    )

    data[post.id] = post_data

    for i, img in enumerate(faces):
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        cv2.imwrite(str(image_path / f"{post.id}-{i}.jpg"), img)

    if j % SAVE_EVERY:
        save_data(data_path, data)


Rateme 2009-12-14 19:50:57: : 97142it [21:01:06,  3.20it/s]

In [11]:
save_data(data_path, data)